In [ ]:
import cv2
import os
import mediapipe as mp
import time

In [ ]:
# --- A. CONFIGURATION ---
# Initialize MediaPipe hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, min_detection_confidence=0.5)

In [ ]:
# List of letters to capture (excluding J and Z for static ASL)
letters = 'TX'
NUM_IMAGES_PER_CLASS = 200 # Total images to capture for each letter
DATASET_DIR = 'asl_dataset'

In [ ]:
# --- B. ROI DEFINITION (CRITICAL FOR TRAINING CONSISTENCY) ---
# Define the fixed Region of Interest (ROI) boundaries on the right side of the screen
# Coordinates (normalized to a 640x480 frame for drawing)
ROI_START_X, ROI_START_Y = 400, 50 
ROI_END_X, ROI_END_Y = 600, 350
ROI_COLOR = (255, 0, 0) # Blue for the ROI box

In [ ]:
if not os.path.exists(DATASET_DIR):
    os.makedirs(DATASET_DIR)

--- C. CAPTURE FUNCTION ---

In [ ]:
def capture_images_for_letter(letter, num_images=NUM_IMAGES_PER_CLASS):
    letter_dir = os.path.join(DATASET_DIR, letter)
    if not os.path.exists(letter_dir):
        os.makedirs(letter_dir)

    cap = cv2.VideoCapture(0)
    # Set standard frame size for consistent ROI drawing
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
    
    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return

    print(f"\n--- Ready to capture {letter}. Hold sign and press 'S' ---")

    count = 0
    while count < num_images:
        ret, frame = cap.read()
        if not ret: break
        
        # Flip the frame and get dimensions
        frame = cv2.flip(frame, 1)
        h, w, _ = frame.shape
        
        # Draw the Fixed ROI Box
        cv2.rectangle(frame, (ROI_START_X, ROI_START_Y), (ROI_END_X, ROI_END_Y), ROI_COLOR, 2)
        
        # Display status
        status_text = f"Class: {letter} | Captured: {count}/{num_images}"
        cv2.putText(frame, status_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
        # Convert to RGB and process with MediaPipe
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(frame_rgb)
        
        # --- LOGIC TO CHECK ROI AND CROP ---
        
        hand_detected_in_roi = False
        
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # 1. Calculate the center of the detected hand
                x_center_normalized = sum(lm.x for lm in hand_landmarks.landmark) / len(hand_landmarks.landmark)
                y_center_normalized = sum(lm.y for lm in hand_landmarks.landmark) / len(hand_landmarks.landmark)
                
                x_center = int(x_center_normalized * w)
                y_center = int(y_center_normalized * h)
                
                # 2. Check if the hand center is within the Fixed ROI box
                if (ROI_START_X < x_center < ROI_END_X) and (ROI_START_Y < y_center < ROI_END_Y):
                    
                    # Hand is correctly positioned in the box
                    hand_detected_in_roi = True
                    
                    # Determine the bounding box coordinates (similar to your old logic)
                    x_min = int(min(lm.x for lm in hand_landmarks.landmark) * w)
                    x_max = int(max(lm.x for lm in hand_landmarks.landmark) * w)
                    y_min = int(min(lm.y for lm in hand_landmarks.landmark) * h)
                    y_max = int(max(lm.y for lm in hand_landmarks.landmark) * h)
                    
                    # Apply margin and clip to frame boundaries
                    box_margin = 20
                    x_min = max(x_min - box_margin, 0)
                    x_max = min(x_max + box_margin, w)
                    y_min = max(y_min - box_margin, 0)
                    y_max = min(y_max + box_margin, h)
                    
                    # Draw a GREEN box for confirmation
                    cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
                    
                    # Crop the hand region
                    hand_region = frame[y_min:y_max, x_min:x_max]
                    
                    # --- SAVE ACTION ---
                    img_path = os.path.join(letter_dir, f'{letter}_{count}_{time.time()}.jpg')
                    cv2.imwrite(img_path, hand_region)
                    count += 1
                    break # Only save one hand per frame

        # Draw status message if outside ROI
        if not hand_detected_in_roi:
            cv2.putText(frame, "MOVE HAND INTO BLUE BOX", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            
        cv2.imshow('ASL Data Collector', frame)

        # Break the capture loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    print(f"Captured {count} images for letter {letter}.")

In [ ]:
# --- D. LOOP EXECUTION ---
if __name__ == '__main__':
    for letter in letters:
        # Before starting capture for the next letter, wait for user confirmation
        print(f"\nReady for letter {letter}. Press Enter in the terminal to begin capturing.")
        input() # Wait for user input in the console
        capture_images_for_letter(letter)

    print("Dataset creation complete!")